##Usar a minha drive



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

#Comentar este pedaço de código para puder correr o código noutro computador

Mounted at /content/gdrive


##Caminho para o dataset na drive

In [ ]:
%cd /content/gdrive/My Drive/ESC-50

#Comentar este pedaço de código para puder correr o código noutro computador

/content/gdrive/My Drive/ESC-50


##Carrega os dados

In [ ]:
import os
import pandas as pd

# Caminho para o dataset
ESC_50_AUDIO_PATH = "/content/gdrive/My Drive/ESC-50/Audio"
ESC_50_METADATA_PATH = "/content/gdrive/My Drive/ESC-50/meta/esc50.csv"

#Trocar o caminho que está para aceder aos audios e os metadados para o caminho próprio

# Carregar metadados
esc50_metadata_df = pd.read_csv(ESC_50_METADATA_PATH)

# Visualizar as primeiras linhas
print(esc50_metadata_df.head())

            filename  fold  target        category  esc10  src_file take
0   1-100032-A-0.wav     1       0             dog   True    100032    A
1  1-100038-A-14.wav     1      14  chirping_birds  False    100038    A
2  1-100210-A-36.wav     1      36  vacuum_cleaner  False    100210    A
3  1-100210-B-36.wav     1      36  vacuum_cleaner  False    100210    B
4  1-101296-A-19.wav     1      19    thunderstorm  False    101296    A


##Carregar a deep neural network

In [ ]:
import tensorflow_hub as hub

# Carregar o modelo YAMNet pré-treinado
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')

##Definição da função de extração

In [ ]:
import librosa
import numpy as np

def extract_yamnet_features(audio_path):
    # Carregar o áudio
    waveform, sample_rate = librosa.load(audio_path, sr=16000)  # YAMNet requer 16 kHz

    # Extrair embeddings com YAMNet
    embeddings, _, _ = yamnet_model(waveform)

    # Tirar a média dos embeddings ao longo do tempo
    return np.mean(embeddings.numpy(), axis=0)

In [ ]:
esc50_metadata_df['file_path'] = esc50_metadata_df.apply(
    lambda row: os.path.join(ESC_50_AUDIO_PATH, row['filename']), axis=1
)

##Extração dos embeddings

In [ ]:
from tqdm import tqdm

features = []
labels = []

for index, row in tqdm(esc50_metadata_df.iterrows(), total=len(esc50_metadata_df)):
    file_path = row['file_path']
    target = row['target']  # Classe do som

    # Extrair features com YAMNet
    yamnet_features = extract_yamnet_features(file_path) # extrai os embeddings dos audios do dataset ESC-50

    features.append(yamnet_features) # coloca os embeddings dos audios em features
    labels.append(target) # coloca  os rotulos dos audios em labels

100%|██████████| 2000/2000 [28:35<00:00,  1.17it/s]


In [ ]:
features_df = pd.DataFrame(features)
features_df['label'] = labels

##Classificador

##Divide os dados para treinamento

In [ ]:
#Nao correr esta celula se for para correr novo dataset de testing.
from sklearn.model_selection import train_test_split

X = features_df.iloc[:, :-1].values  # Features extraídas do aúdio (são os embeddings gerados pelo yamnet. Cada linha de X é um vetor de 1024 )
y = features_df['label'].values     # Labels(rótulos associados a cada som no dataset ESC-50)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#Modelo é treinado em 80% dos dados-> (x_train e y_train) e avaliado nos outros 20% ->(X_test e Y_test)

NameError: name 'features_df' is not defined

##Correr apenas se for para testar novo dataset de testing

In [ ]:
#Correr esta celula e não a de cima se para testar novo dataset de testing
from sklearn.model_selection import train_test_split

X_test = features_df.iloc[:, :-1].values  # Features extraídas do audio do novo dataset (são os embeddings gerados pelo yamnet. Cada linha de X é um vetor de 1024 )
y_test = features_df['label'].values     # Labels(rótulos associados a cada som no dataset de testing)


##Treina o modelo

In [ ]:
#Não treinar o modelo com o novo dataset, apenas avaliar a performance.Assim usamos o aprendizado do modelo usando o ESC-50
#Logo, comentar ou não correr a célula seguinte ao avaliar o modelo

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=100, random_state=42) #cria uma floresta de 100 arvores de decisão e com mesma semente
classifier.fit(X_train, y_train) # treina o random florest com os dados de treino x_train e os embeddings das classes associadas
#Durante o treinamento cada árvore é ajustada para aprender padrões e regras que associem embeddings com as classes dos sons
#Cria árvores usando amostras aleatórias de X_train e subconjuntos de características para criar diversidade nas árvores

NameError: name 'X_train' is not defined

##Prever e avaliar as previsões

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = classifier.predict(X_test) #usa o modelo treinado para prever os rotulos das amostras em X_test que contem os embeddings
#y_pred são as classes previstas pelo modelo para o conjunto de embeddings dos dados de testes fornecidos
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.82
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      0.83      0.91         6
           2       1.00      0.67      0.80         9
           3       0.93      1.00      0.96        13
           4       0.91      0.91      0.91        11
           5       0.92      1.00      0.96        12
           6       0.92      1.00      0.96        12
           7       0.86      1.00      0.92         6
           8       1.00      1.00      1.00        10
           9       1.00      0.83      0.91         6
          10       0.80      0.80      0.80         5
          11       0.91      0.83      0.87        12
          12       1.00      0.67      0.80         9
          13       1.00      1.00      1.00         6
          14       0.86      1.00      0.92         6
          15       1.00      0.57      0.73        14
          16       0.70      0.78      0.7

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


##Validação cruzada

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=42)
scores = cross_val_score(clf, X, y, cv=5)
print("Scores da validação cruzada:", scores)

#pois algumas classes podem ser descaracterizadas